<a href="https://colab.research.google.com/github/skrzypczykt/MAchineLearningProjects/blob/main/NeuralNetworksTutorials/TFLearn/Zadanie.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Runtime i instalacja

Zacznijmy od włączenia wsparcia obliczeń kartą graficzną (GPU), która znacznie przyspieszy nasze działania. Może się zdażyć, że nie będzie dostępnych GPU w Colaboratory – wtedy można pracować bez niego, jedynie obliczenia potrwają kilka razy dłużej.

Wybieramy w menu **`Runtime`** -> **`Change runtime type`** , a następnie w podmenu **`Hardware accelerator`** wybieramy **`GPU`**.


<img style="float:left" src="https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/images/change-runtime1.png">
<img src="https://www.mimuw.edu.pl/~mm319369/priv/d73890416bec03ff3e2b3756af8c941c/images/change-runtime2.png">

## TFlearn

W tym zadaniu będziemy uczyć sieci neuronowe i obserwować jak zachowują się, gdy zmieniamy liczbę neuronów w sieci, funkcje aktywacji i co się dzieje, gdy sieć jest za duża względem danych. Wasze zadanie będzie polegało na odpowiedniej konfiguracji sieci, a następnie narysowaniu wykresów zaobserwowanych rezultatów.

Będziemy korzystać z biblioteki [*tflearn*](http://tflearn.org/getting_started/), która jest prostym interfejsem do `tensorflow`.

Poniżej ją instalujemy i importujemy potrzebne biblioteki.

In [ ]:
!pip install tflearn "tensorflow<2"

In [ ]:
from __future__ import division, print_function, absolute_import

import time
import math

import tflearn
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.normalization import local_response_normalization
from tflearn.layers.estimator import regression
import tensorflow as tf
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

# Monkey-patchujemy tflearn, żeby nie wypisywał samemu statystyk,
# któe nie działają dobrze w colaboratory
tflearn.callbacks.TermLogger.print_termlogs = lambda *args, **kwargs: None

# Zbiór danych (dataset)

Będziemy pracować na już tradycyjnym zbiorze – rozpoznawania odręcznie pisanych cyfr [MNIST](http://yann.lecun.com/exdb/mnist/).
Elementy tego zbioru to obrazki 28x28 pikseli w skali szarości, a ich etykietami są cyfry im odpowiadające. Mamy 55000 przykładów treningowych i 10000 przykładów testowych.
Dla wydajności obliczeń, wszystkie dane są umieszczane w dużych macierzach (tablicach).

Poniższa komórka pobiera i ładuje zbiór danych do pamięci. Wyjaśnienie wprowadzonych zmiennych poniżej.


In [ ]:
# Tu ładujemy nasz dataset - zestaw odręcznie pisanych cyfr od 0 do 9


import tflearn.datasets.mnist as mnist
X, Y, testX, testY = mnist.load_data(one_hot=True) # ładowanie datasetu (automatycznie ściąga się z internetu)

# X,Y to dane treningowe
# testX, textY to dane walidacyjne - sprawdzamy nimi jak dobra jest nasza sieć

# nasze obrazy mają mieć wymiar 28x28 pikseli - będziemy używać sieci konwolucyjnych, więc konieczna 
# jest zmiana wymiarów macierzy poleceniem reshape
# [samples, x_size, y_size, channels]
# samples - liczba próbek 
# x_size, y_size - wymiary obrazka
# channels - liczba kolorów/kanałów - w naszym przypadku 1, bo obrazki są czarno-białe
X = X.reshape([-1, 28, 28, 1])
testX = testX.reshape([-1, 28, 28, 1])

**X** oraz **testX** to 4-wymiarowe macierze z przykładami odpowiednio treningowymi i testowymi. Wymiary tych macierzy to \[*liczba przykładów*, *rozmiar x*, *rozmiar y*, *liczba kolorów*]. Różne funkcje oczekują odpowiednich wymiarów macierzy. Nie będzie to na szczęście dla nas istotne poza zrozumieniem napisów, które się pojawiają:

`X[4]`  - przykład numer 4 (wymiar [28, 28, 1]) \\
`X[2:3]`  - przykład numer 2, ale w wymiarze [1, 28, 28, 1] - odpowiednie dla funkcji/metod tflearn \\
`X[:10]`  - przykłady numer 0...9, w wymiarze [10, 28, 28, 1] \\
`X[2,:,:,0]` - przykład 2 w w wymiarze [28, 28] odpowiednim dla funkcji do rysowania

**Y** oraz **testY** to 2-wymiarowe macierze z etykietami odpowiednio treningowymi i testowymi. Wymiary tych macierzy to \[*liczba przykładów*, *liczba cyfr*]. Te macierze reprezentują tzw. kodowanie *one-hot* etykiet  - dany przykład przedstawiający cyfrę **i** ma **1** na **i**-tej pozycji, a na pozostałych **0**.

`Y[0]` - etykieta przykładu numer 0 (wymiar [10]) \\
`Y[2:3]` - etykieta przykładu numer 2, ale w wymiarze [1, 10] - odpowiednie dla funkcji/metod tflearn \\
`Y[:10]` - etykiety dla przykładów numer 0..9 (j.w.)

Poniżej przykład użycia na przykładach numer 0 i 1 (powinny reprezentować 7 i 3).

In [ ]:
# Kilka przykładów z naszego datasetu
print(Y[0]) # etykieta - 1 tam, gdzie poprawna klasa
plt.imshow(X[0,:,:,0], cmap='gray') # reprezentacja graficzna obrazka
plt.show()

print(Y[1])
plt.imshow(X[1,:,:,0], cmap='gray')
plt.show()

# Budowa sieci

Zdefiniujmy najpierw klasę pomocniczną, która będzie przetrzymywać wyniki obliczeń i przyda nam się do wykresów. Opiszemy ją później.

In [ ]:
# Definiujemy klasę, która będzie przetrzymywać wyniki obliczeń po każdej epoce
class Stats(tflearn.callbacks.Callback):
    def __init__(self, examples=0):
        self.epoch_data = []
        self.data_size = examples
        self.last_time = 0
        self.start_time = 0
        
        self.min_report_secs = 1.

    def on_train_begin(self, training_state):
        self.start_time = time.time()
      
    def on_epoch_end(self, training_state):
        metrics = {
            'loss': training_state.loss_value, # loss (strata) dla danych treningowych
            'acc': training_state.acc_value,   # accuracy (dokładność) dla danych treningowych
            'val_loss': training_state.val_loss, # loss dla danych walidacyjnych
            'val_acc': training_state.val_acc, # accuracy dla danych walidacyjnych
            'epoch': training_state.epoch,
            'step': training_state.step,
            'iter': training_state.current_iter,
            'time': time.time() - self.start_time,
        }

        self.epoch_data.append(metrics)
        
    def on_batch_end(self, training_state, snapshot=False):
      cur_time = time.time()
      if cur_time - self.last_time < self.min_report_secs:
        return # nie spamujmy za często
      
      self.last_time = cur_time
      epoch = training_state.epoch
      step = training_state.step
      iter = training_state.current_iter
      print("Epoch %d, step (batch no.): %d -- acc: %.2f, loss %.2f -- iter %05d/%05d, training for: %.2fs" % (
          epoch, step, training_state.acc_value, training_state.loss_value,
          iter, self.data_size, cur_time-self.start_time))

\\

Poniżej jest przykład budowy sieci przy korzystaniu z `tflearn`. W zadaniu będzie trzeba zmieniać odpowiednie parametry.

Sieć budujemy kolejnymi warstwami, gdzie funkcja tworząca następną warstwę przyjmuje zmienną reprezentującą poprzednią. Dla wygody wynik przypisujemy na tą samą zmienną, aby móc łatwo dodawać dodatkowe warstwy poprzez wklejenie kodu w odpowiednie miejsce:

`network = ...` <- opis warstwy A \\
`network = conv_2d(network, ...)` <- tworzymy nową warstwę B za warstwą A i przypisujemy na tą samą zmienną (nie potrzebujemy już odwoływać się do warstwy A) \\
`network = fully_connected(network, ...)` <- warstwa C za warstwą B

W tym momencie zmienna `network` reprezentuje sieć składającą się z warstw `A-B-C`.

### Uczenie

Uczenie sieci przebiega w fazach. Podajemy sieci przykłady kolejno i stosujemy propagację wstęczną. W momencie, gdy przykłady nam się skończą, po prostu zaczynamy od nowa. Każdy taki obrót będziemy nazywali **epoką**.  \\
Przykładów nie będziemy podawać sieci pojedynczo – podajemy je w porcjach rozmiaru `batch` (*wsad*), co oznacza ile przykładów sieć będzie przetwarzać jednocześnie.

Przydatne nam będą przede wszystkim funkcje:

`conv_2d(network, #filtrów, #rozmiar, activation=f.aktywacji, regularizer="L2")` - tworzy nową warstwę konwolucyjną posiadającą `#filtrów` filtrów (neuronów) o rozmiarze `#rozmiar x #rozmiar` oraz z funkcją aktywacji jak podana. Wartości `regularizer` nie należy zmieniać w tym zadaniu.

`max_pool_2d(network, #rozmiar)` - podpróbkowanie danych co `#rozmiar` (zmniejsza wymiar obrazka razy `#rozmiar`)

`fully_connected(network, #neuronów, activation=f.aktywacji)` - warstwa pełna  z `#neuronów` neuronów i podaną funkcją aktywacji.

**W kodzie zaznaczono, który fragment będzie podlegał modyfikacji.**

In [ ]:
# Tu definiujemy architekturę naszej sieci

# !!! UWAGA - przy każdych nowych obliczeniach należy zresetować graf sieci
tf.reset_default_graph()

# Wyłączamy warningi z tensorflow
tf.logging.set_verbosity(tf.logging.ERROR)


# Warstwa wejściowa - musi mieć takie same wymiary jak dane
network = input_data(shape=[None, 28, 28, 1], name='input') # None oznacza, że ta 
# wartość będzie uzupełniona automatyzcnie i jest to liczba próbek we wsadzie (batch)

### MODYFIKUJEMY OD TĄD
# ---
# Pierwsza warstwa konwolucyjna - 32 filtry o rozmiarach 3x3
# z funkcją aktywacji (activation=) relu.
# Regularizer='L2' oznacza narzucenie ograniczenia na wartości wag (nie istotne tutaj).
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
# max_pool wykonuje podpróbkowanie danych - zmniejsza wymiar obrazka 2-krotnie
network = max_pool_2d(network, 2) # teraz obrazki są [14x14]

# druga warstwa konwolucyjna - 32 filtry o rozmiarach 3x3 z podpróbkowaniem
network = conv_2d(network, 32, 3, activation='relu', regularizer="L2")
network = max_pool_2d(network, 2) # teraz obrazki są [7x7]

# warstwa pełna - tu już zaczyna się "normalna" sieć neuronowa
network = fully_connected(network, 128, activation='relu') # 128 neuronów, aktywacja "relu", przyjmuje wejście [7x7] do [128] neuronów
network = fully_connected(network, 256, activation='relu') # 256 neuronów, aktywacja "relu"
# ---
### DO TĄD

# warstwa wyjściowa - używamy aktywacji softmax, żeby dostać prawdopodobieństwa dla każdej klasy (cyfry)
network = fully_connected(network, 10, activation='softmax')

# tu definiujemy w jaki sposób optymalizować sieć (regression nie oznacza, że robimy regresję)
# nie będziemy modyfikować tych argumentów; stosujemy optymizator Adam
network = regression(network, optimizer='adam', learning_rate=0.01,
                     batch_size=250,
                     loss='categorical_crossentropy', name='target')

Powyżej zdefiniowaliśmy sieć z warstwą wejściową, dwiema warstwami konwolucyjnymi z 32 filtrami 3x3 i podpróbkowaniem x2 oraz trzema warstwami pełnymi z odpowiednio 128, 256 i 10 neuronami.

## Trenowanie sieci

Trenowanie sieci wywołujemy jak poniżej (wszystkie 3 linijki będą potrzebne). Trenowanie możemy w dowolnym momencie przerwać klikając na kwadrat zatrzymywania.

Metoda uczenia ma następującą sygnaturę:
`model.fit({'input': X}, {'target': Y}, n_epoch=#epok,
           validation_set=({'input': testX}, {'target': testY}), show_metric=True, callbacks=[scores])`
  
 Uczy podaną wyżej sieć, z danymi treningowymi z `X` i etykietami z `Y` przez `#epok` epok. Wyniki walidacji będą obliczane na zbiorze testowym `testX` z etykietami `testY`. Resztę zostawiamy bez zmian – służy do wypisywania i rejestrowania statystyk.

Podczas uczenia będziemy zbierali miary tego, jak sieć radzi sobie z naszymi danymi:

 `loss` - loss (strata) dla danych treningowych \\
`acc` - accuracy (dokładność) dla danych treningowych \\
`val_loss` -  loss dla danych walidacyjnych \\
`val_acc` - accuracy dla danych walidacyjnych \\
`epoch` - numer *epoki* \\
`step` - numer wsadu (*batch*) \\
`iter` - liczba widzianych przykładów \\
`time` - czas jaki upłynął od początku uczenia \\
  

**Uwaga!** kolejne wywołania poniższej komórki będą *douczać* sieć – aby uczyć ją od nowa musimy ponownie wykonać komórkę z definicją sieci.

In [ ]:
# uruchamiamy trenowanie naszej sieci
# n_epoch - liczba epok, czyli przejść przez cały zbiór danych treningowych
scores = Stats(examples=len(X))
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X}, {'target': Y},
          n_epoch=20,  # <-- DO ZMIANY
          validation_set=({'input': testX}, {'target': testY}), show_metric=True, callbacks=[scores])


## Korzystanie z sieci

Poniźej przykład korzystania z sieci do etykietowania danych które sieć nie widziała.

In [ ]:
# Wylosujmy index elementu ze zbioru testowego do sprawdzenia
test_index = np.random.randint(len(testX))
# Przewidywania modelu na danych walidacyjnych (prawdopodobieństwa/logika rozmyta):
predictions = model.predict(testX[test_index:test_index+1])
plt.imshow(testX[test_index,:,:,0], cmap='gray')
plt.show()
print(predictions)
print("Prawdopodobnie: %s na %.3f%%" % (predictions.argmax(), predictions.max()*100))

# Programatyczny dostęp do statystyk

Statystyki znajdziemy w obiekcie `scores`, który przekazaliśmy podczas uczenia sieci. Jest to lista statystyk po każdej *epoce*. Statystyki reprezentowane są przez słownik z polami jak opisanymi wcześniej.

In [ ]:
# wyświetlenie danych
scores.epoch_data

# POLECENIA
## Do zaliczenia tego zadania można zrobić podzadania A-C lub zadanie alternatywne.

# Uwagi:

### Przy każdych nowych obliczeniach należy zresetować graf sieci.
### Nie należy zmieniać linijki z optymizatorem przy rozwiązywaniu zadań.
### Niektóre rezultaty mogą być nieintuicyjne.

# Zadanie A: dawne ograniczenia

Narysuj wykres zależności accuracy dla danych **walidacyjnych** (val_acc) od liczby epok (max 10 epok)
dla sieci w dwóch wariantach:
* 1 warstwa konwolucyjna: 32 filtry 3x3 + 2 warstwy pełne (jak w przykładzie: 128,256,10)
* j.w. ale dla 3 i 5 warstw konwolucyjnych (podpróbkowanie wykonuj tylko po 2. i 4. warstwie konwolucyjnej)

# Zadanie B - funkcje aktywacji

Narysuj wykres zależności accuracy dla danych **walidacyjnych** (val_acc) od liczby epok, dla sieci konwolucyjnej: 
32-3x3, podpróbkowanie x2, 32-3x3, podpróbkowanie x2, 32-3x3, z wartstwami pełnymi: 128, 256, 10

(przez 32-3x3 rozumiemy warstwę konwolucyjną z 32 filtrami rozmiaru 3x3)

w zależności od funkcji aktywacji (ustawianej w parametrze `activation=`):
* `'relu'`
* `'sigmoid'`
* `'elu'`

** Uwaga! ** Nie należy zmieniać funkcji aktywacji `softmax` w ostatniej warstwie.

# Zadanie C: overfitting
Narysuj wykres zależności accuracy dla danych **treningowych** (acc) i **walidacyjnych** (val_acc) od liczby epok (50 epok), dla sieci która ma tylko jedną ukrytą pełną warstwę z 1000 neuronów z aktywacją `relu` i warstwę wyjściową z 10 neuronami. 

Obliczenia wykonaj używając tylko 1000 pierwszych próbek z zestawu treningowego, jak pokazano poniżej:

In [ ]:
scores = Stats(1000)
model = tflearn.DNN(network, tensorboard_verbose=0)
model.fit({'input': X[:1000]}, {'target': Y[:1000]}, n_epoch=50,
           validation_set=({'input': testX}, {'target': testY}), show_metric=True, callbacks=[scores])


# Przesyłanie rozwiązań

Zadania należy wysyłać na adresy `slezak@mimuw.edu.pl` i `m.matraszek@mimuw.edu.pl`  z tytułem o przedrostku [SI20-3]. 
Przesłane wykresy mogą być w dowolnej formie. Mile widziany byłby odpowiednio zmieniony ten notebook. Poniżej można znaleźć prosty sposób rysowania wykresów.

In [ ]:
# Przykład użycia matplotlib do rysowania wykresów:

# Wartości w punktach [0, 1, 2, ...]; podajemy tylko Y
plt.plot([0, 1, 1.5, 6], label="asd")
# X podany w pierwszym argumencie, Y w drugim
plt.plot([0, 2, 6], [7, 0, 0.6], label="bbb")
# rysuje legendę (linie podpisane jako `label`)
plt.legend()

# Zadanie alternatywne

Zamiast rozwiązywania powyższych zadań, proponujemy też kontynuację przygody z OpenAI Gym.
W tej wersji chcielibyśmy użyć dość prostej metody łączącej uczenie ze wzmocnieniem i sieci neuronowe. Jest to Policy Gradient. W miarę przystępny opis tej metody jest zamieszczony tutaj: http://karpathy.github.io/2016/05/31/rl/

Używając tflearn z tą metodą, rozwiąż środowisko CartPole-v0 z OpenAI Gym. A może nawet coś trudniejszego?
Wykonanie:
```python
model.fit({'input': X}, {'target': Y}, n_epoch=1)
```
pozwoli na jednokrotną iterację propagacji wstecznej na podanych danych. Metodę tę można wykonać ponownie z innymi danymi.